In [2]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.linear_model import SGDClassifier
from nltk.corpus import stopwords
import string

In [3]:
data = pd.read_parquet('../input/....')

In [4]:
train = data[~data.category_id.isin({-1, 121})].drop_duplicates('item_name')
print(train.shape)

(48225, 9)


In [6]:
import re

def preprocess(text):
    text = text.lower()
    text = "".join(c if c not in string.punctuation else f" {c} " for c in text )
    return " ".join(w.strip() for w in text.split())

def remove_numbers(text):
    result = re.sub(r'\d+', '', text)
    return result

def remove_punctuation(text):
    translator = str.maketrans('', '', '!"#$&%*\'()+.,:;<=>?@[\\]^_`{|}~№')
    return text.translate(translator)

def remove_slash(text):
    translator = str.maketrans('', '', '/-')
    return text.translate(translator)


def remove_prob(text):
    result = re.sub(r'  *', ' ', text)
    return result

def remove_letter(text):
    text = " ".join(word for word in text.split() if len(word)>2)
    return text

#def replace_text(text):
#    for i in valc.keys():
#        result = re.sub(r"\b(%s)\b" % (i), r"%s" % (valc[i]), text)
#        return result

#    result = str.replace(u'  *',' ')
#    return result
#    frm = ''.join([i for i in valc.keys()])
#    to = ''.join([i for i in valc.values()])
#    result = re.sub(frm, to, text)
#    result = re.sub(r"\b(%s)\b" % frm, r" %s " % to, text)
#    trantab = str.maketrans(frm, to)
#    return text.translate(trantab)
#    return result

#def prep(text_feature):
#    text_feature = text_feature.apply(replace_text)
#    return text_feature

def preprocessing_text(text_feature):
    text_feature = text_feature.apply(preprocess)
#    text_feature = text_feature.apply(remove_numbers)
    text_feature = text_feature.apply(remove_punctuation)
    text_feature = text_feature.apply(remove_prob)
#    text_feature = text_feature.apply(remove_letter)
    return text_feature

def preprocessing_text_after_transcripting(text_feature):
    text_feature = text_feature.apply(remove_slash)
#    text_feature = text_feature.apply(remove_letter)
    text_feature = text_feature.apply(remove_prob)
    return text_feature

In [7]:
train['item_name'] = preprocessing_text(train['item_name'])
#print(train.item_name.to_string())

In [8]:
sokr = {
r'\bпорц\b': 'порция',
r'\bнап\b': 'напиток',
r'\bн - к\b': 'напиток',    
r'\bмол\b': 'молочный',
r'\bмолоч\b': 'молочный',
r'\bваф\b': 'вафельный',
r'\bпюр\b': u'пюре',
r'\bдет\b': u'детское',
r'\bстер\b': u'стерилизованное',
r'\bкотл\b': u'котлеты',
r'\bкол\b': u'колбаса',
r'\bсос\b': u'сосиски',
r'\bсосис\b': u'сосиски',    
r'\bпаш\b': u'паштет',
r'\bколб\b': u'колбаса',
r'\bпельм\b': u'пельмени',    
r'\bпломб\b': u'пломбир', 
    
r'\bхл\b': u'хлопья',
r'\bовс\b': u'овсяные',
r'\bмак\b': u'макароны',
r'\bшок\b': u'шоколадный',
r'\bшокол\b': u'шоколадный',
r'\bшоколадн\b': u'шоколадный',    
r'\bкф\b': u'конфеты',
r'\bконф\b': u'конфеты',
r'\bкоф\b': u'кофе',
r'\bконд\b': u'кондиционер',
r'\bчист\b': u'чистящее',
r'\bнак\b': u'накаливания',
r'\bкух\b': u'кухонная',
r'\bкош\b': u'кошек',
r'\bнаполн\b': u'наполнитель',
    
r'\bнар\b': u'нарезной',
r'\bнарез\b': u'нарезной',
r'\bэнерг\b': u'энергетический',
r'\bэнергет\b': u'энергетический',
r'\bслив\b': u'сливочный',
r'\bчер\b': u'черный',
r'\bзел\b': u'зеленый',
r'\bшт\b': u'штук',
#r'\bг\b': u'грамм',
r'\bгр\b': u'грамм',
r'\bмг\b': u'миллиграмм',
r'\bкг\b': u'килограмм',
r'\bмл\b': u'миллилитр',
r'\bваф\b': u'вафли',
r'\bпеч\b': u'печенье',   
r'\bкф\b': u'конфеты',
r'\bпри-ва\b': u'приправа',   

r'\bсл\b': u'сладкое',
r'\bсух\b': u'сухое',
r'\bбут\b': u'бутылка',
r'\bпаст\b': u'пастеризованное',
r'\bразл\b': u'разливное',  
r'\bст / б\b': u'стеклянная банка',
r'\bс / б\b': u'стеклянная банка',
r'\bж / б\b': u'жестянная банка',
r'\bпл / б\b': u'пластиковая бутылка',
r'\bп / сл\b': u'полусладкое',
r'\bн / ф\b': u'нефильтрованное', 
r'\bсветл\b': u'светлое',     
    
r'\bдиз\b': u'дизельное',
r'\bдизтопиво\b': u'дизельное топливо',
r'ж - л': u'журнал',
r'\bжур\b': u'журнал',
r'\bпласт\b': u'пластик',
r'\bруч\b': u'ручка',
r'\bаром\b': u'ароматический',
r'\bароматич\b': u'ароматический',

r'\bмуж\b': u'мужской',
r'\bжен\b': u'женский',
r'\bодек\b': u'одеколон',
r'\bвол\b': u'волос',
r'\bтрик\b': u'трикотажная',
r'\bгов\b': u'говядина',
r'\bкур\b': u'курица',
r'\bфил\b': u'филе',    
r'\bохл\b': u'охлажденный',
r'\bсалф\b': u'салфетки',
    
   
r'\bд / п\b': u'детское питание',
    
r'\bс / к\b': u'сырокопченая колбаса',
r'\bс / в\b': u'сыровяленая колбаса',
#r'\bп / к\b': u'полукопченая колбаса',
r'\bв / к\b': u'варенокопченая колбаса',
r'\bвар\b': u'вареная',    
r'\bжар\b': u'жареная',  
    
r'\bср - во\b': u'средство', 
r'\bг / д / д\b': u'гель для душа',
r'\bд /\b': u'для',
#r'трк': u'топливораздаточная колонка',
#r'грк': u'газораздаточная колонка',
r'\bдт\b': u'дизельное топливо',
r'\bаи': u'бензин',
r'\bпбт': u'топливо',
r'\bпвх': u'пластик',
r'\bпэт': u'пластик',  
    
r'\bр - р': u'раствор',
r'\bсуспб\b ': u'суспензия',
r'\bп / об\b ': u'таблетка', 
r'\bтаб\b': u'таблетка',
r'\bтбл\b': u'таблетка',
r'\bжев\b': u'жевательные',
r'\bкапс\b': u'капсула',
r'\bфл\b': u'флакон',  
r'\bмед\b': u'медицинский',
r'\bт п о\b': u'таблетка',
r'\bинг\b': u'ингаляция',
    
r'\bз / щ\b': u'зубная щетка', 
r'\bз / щетка\b': u'зубная щетка', 
r'\bз / паста\b': u'зубная паста',     
r'\bт / в\b': u'туалетная вода',
r'\bосвеж\b': u'освежитель',
r'\bз / м\b': u'замороженный',
r'\bгвс\b': u'водоснабжение',        
r'\bхвс\b': u'водоснабжение',  

r'\bж / р\b': u'жевательная резинка',
r'\bрез\b': u'резинка',
r'\bжев\b': u'жевательная',
r'\bжеврез\b': u'жевательная резинка',    

r'\bцб\b': u'цыплят бройлеров',
r'\bцып\b': u'цыплят',

r'\bкр\b': u'крем',
r'\bбум\b': u'бумага',
r'\bкпб\b': u'комплект постельного белья',    
r'\bмин\b': u'минеральная',    
r'\bнегаз\b': u'негазированная', 
r'\bм - з\b': u'майонез',
r'\bч / с\b': u'чистящее',
    
    
r'\bkофе\b': u'кофе',
r'\bkотлета\b': u'котлета',
r'\bkуриная\b': u'куриная',
r'\bkартофель\b': u'картофель',
r'\bkекс\b': u'кекс',
r'\bkонфеты\b': u'конфеты',    
r'\bмясоkраба\b': u'мясо краба',
r'\bkаша\b': u'каша',
r'\bkиви\b': u'киви',
    
r'ё': u'е',
r'\bемк\b': u'емкость',
r'\bтетр\b': u'тетрадь',
r'\bкан\b': u'канцелярия',
r'\bпрокл\b': u'прокладки',
r'\bг / п\b': u'прокладки',
r'\bзапрп\b': u'заправка',
r'\bмеш\b': u'мешок',
r'\bмус\b': u'мусор',
r'\bш к о\b': u'шоколад'
}


In [9]:
for i in sokr.keys():
    train['item_name'] = train['item_name'].str.replace(r"%s" % (i), r"%s" % (sokr[i]))

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:2: FutureWarning: The default value of regex will change from True to False in a future version.
  


In [10]:
train['item_name'] = preprocessing_text_after_transcripting(train['item_name'])